In [1]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install tiktoken
!pip install langchain_experimental
!pip install wikipedia
!pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requ

In [2]:
%%writefile qwen.env
## Qwen base_url
OPENAI_BASE_URL='https://dashscope.aliyuncs.com/compatible-mode/v1'
DASHSCOPE_API_KEY='sk-d4c3f773af5d442c97eb7d9e40479c55'
OPENAI_API_KEY=${DASHSCOPE_API_KEY}

Writing qwen.env


In [3]:
## 使用qwen key
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('qwen.env',override=True)

import os
print(os.environ['OPENAI_BASE_URL'],os.environ['OPENAI_API_KEY'])

https://dashscope.aliyuncs.com/compatible-mode/v1 sk-d4c3f773af5d442c97eb7d9e40479c55


## langchain.agents

### 核心框架
ReAct:https://react-lm.github.io/

### tools
每个tool需要有三个属性：
- name
- description
- func
description和name主要用于提示词，func用于调用。


In [4]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool

from langchain.chat_models import ChatOpenAI

model_name='qwen2-7b-instruct'
llm = ChatOpenAI(model=model_name,temperature=0)


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [19]:
tools = load_tools(
    ["llm-math","wikipedia"], 
    llm=llm #第一步初始化的模型
)

In [14]:
tools[0].description,tools[0].name

('Useful for when you need to answer questions about math.', 'Calculator')

In [20]:
# 初始化代理
agent= initialize_agent(
    tools, #第二步加载的工具
    llm, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)
agent("计算300的25%") 

In [22]:
question = "Tom M. Mitchell是一位美国计算机科学家，\
也是卡内基梅隆大学（CMU）的创始人大学教授。\
他写了哪本书呢？"

agent(question) 




> Entering new AgentExecutor chain...
Thought: 我需要查询 Tom M. Mitchell 的著作。
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


/opt/conda/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/conda/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtai

{'input': 'Tom M. Mitchell是一位美国计算机科学家，也是卡内基梅隆大学（CMU）的创始人大学教授。他写了哪本书呢？',
 'output': 'Tom M. Mitchell 写了《Machine Learning》这本书。'}

## 自定义工具
使用tool装饰器装饰func函数，并在函数内放description即可

#### 计算日期工具

In [8]:
# 导入tool函数装饰器
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """
    返回今天的日期，用于任何需要知道今天日期的问题。\
    输入应该总是一个空字符串，\
    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。
    """
    return str(date.today())

# 初始化代理
agent= initialize_agent(
    tools=[time], #将刚刚创建的时间工具加入代理
    llm=llm, #初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

# 使用代理询问今天的日期. 
# 注: 代理有时候可能会出错（该功能正在开发中）。如果出现错误，请尝试再次运行它。
agent("今天的日期是？") 




> Entering new AgentExecutor chain...
Thought: 问题要求提供今天的日期，这需要使用到time工具。

Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-08-06
Thought:我已知今天的日期是2024年8月6日。

Final Answer: 今天的日期是2024年8月6日。

> Finished chain.


{'input': '今天的日期是？', 'output': '今天的日期是2024年8月6日。'}

In [10]:
time.description,time.name,time.func

('返回今天的日期，用于任何需要知道今天日期的问题。    输入应该总是一个空字符串，    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。',
 'time',
 <function __main__.time(text: str) -> str>)

#### google_search工具

In [16]:
@tool
def google_search(search_query: str):
    """
    谷歌搜索是一个通用搜索引擎，可用于访问互联网、查询百科知识、了解时事新闻等。\
    输入是搜索关键词或短语，\
    输出是对关键词的检索结果。
    """
    url = "https://google.serper.dev/search"
    import json,requests
    payload = json.dumps({"q": search_query})
    headers = {
        'X-API-KEY': 'd8ba7d116c5dd2154ed7d5ba08caea5b0d4c0197',
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload).json()

    return response['organic'][0]['snippet']

google_search.description

'谷歌搜索是一个通用搜索引擎，可用于访问互联网、查询百科知识、了解时事新闻等。    输入是搜索关键词或短语，    输出是对关键词的检索结果。'

In [18]:
agent= initialize_agent(
    tools=[google_search], #将刚刚创建的时间工具加入代理
    llm=llm, #初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 使用代理询问今天的日期. 
# 注: 代理有时候可能会出错（该功能正在开发中）。如果出现错误，请尝试再次运行它。
agent("Tom M. Mitchell是一位美国计算机科学家，也是卡内基梅隆大学（CMU）的创始人大学教授。他写了哪本书呢？") 



> Entering new AgentExecutor chain...
Thought: 我需要找到关于 Tom M. Mitchell 的书籍信息。
Action:
```
{
  "action": "google_search",
  "action_input": "Tom M. Mitchell books"
}
```

Observation: Books · Image of Machine Learning: An Artificial Intelligence Approach (Volume I) · Image of Mind Matters: A Tribute To Allen Newell (Carnegie Mellon Symposia ...
Thought:我找到了关于 Tom M. Mitchell 的书籍信息。其中一本是 "Machine Learning: An Artificial Intelligence Approach (Volume I)"。另一本是 "Mind Matters: A Tribute To Allen Newell (Carnegie Mellon Symposia)"。

Final Answer: Tom M. Mitchell 写了多本书籍，其中包括 "Machine Learning: An Artificial Intelligence Approach (Volume I)" 和 "Mind Matters: A Tribute To Allen Newell (Carnegie Mellon Symposia)"。

> Finished chain.


{'input': 'Tom M. Mitchell是一位美国计算机科学家，也是卡内基梅隆大学（CMU）的创始人大学教授。他写了哪本书呢？',
 'output': 'Tom M. Mitchell 写了多本书籍，其中包括 "Machine Learning: An Artificial Intelligence Approach (Volume I)" 和 "Mind Matters: A Tribute To Allen Newell (Carnegie Mellon Symposia)"。'}

### 也可继承BaseTool类实现自定义工具
类中要使用三个属性：
- name
- description
- return_direct

In [5]:
from langchain.tools import BaseTool, DuckDuckGoSearchRun

# 搜索工具
class SearchTool(BaseTool):
    name = "Search"
    description = "当问电影相关问题时候，使用这个工具"
    return_direct = False  # 直接返回结果

    def _run(self, query: str) -> str:
        print("\n正在调用搜索引擎执行查询: " + query)
        search = DuckDuckGoSearchRun()
        return search.run(query)

# 计算工具
class CalculatorTool(BaseTool):
    name = "Calculator"
    description = "如果问数学相关问题时，使用这个工具"
    return_direct = False  # 直接返回结果

    def _run(self, query: str) -> str:
        return eval(query)



In [9]:
from typing import Dict, Union, Any, List

from langchain.output_parsers.json import parse_json_markdown
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.agents import AgentExecutor, AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
import re
# 自定义解析类
class CustomOutputParser(AgentOutputParser):

    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        print(text)
        cleaned_output = text.strip()
        # 定义匹配正则
        action_pattern = r'"action":\s*"([^"]*)"'
        action_input_pattern = r'"action_input":\s*"([^"]*)"'
        # 提取出匹配到的action值
        action = re.search(action_pattern, cleaned_output)
        action_input = re.search(action_input_pattern, cleaned_output)
        if action:
            action_value = action.group(1)
        if action_input:
            action_input_value = action_input.group(1)
        
        # 如果遇到'Final Answer'，则判断为本次提问的最终答案了
        if action_value and action_input_value:
            if action_value == "Final Answer":
                return AgentFinish({"output": action_input_value}, text)
            else:
                return AgentAction(action_value, action_input_value, text)

        # 如果声明的正则未匹配到，则用json格式进行匹配
        response = parse_json_markdown(text)
        
        action_value = response["action"]
        action_input_value = response["action_input"]
        if action_value == "Final Answer":
            return AgentFinish({"output": action_input_value}, text)
        else:
            return AgentAction(action_value, action_input_value, text)
output_parser = CustomOutputParser()



In [10]:
from langchain.memory import ConversationBufferMemory
from langchain.agents.conversational_chat.base import ConversationalChatAgent 
from langchain.agents import AgentExecutor, AgentOutputParser

SYSTEM_MESSAGE_PREFIX = """尽可能用中文回答以下问题。您可以使用以下工具"""


llm = ChatOpenAI(model_name=model_name, request_timeout=60)
# 初始化工具
tools = [CalculatorTool(), SearchTool()]
# 初始化对话存储，保存上下文
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# 配置agent
chat_agent = ConversationalChatAgent.from_llm_and_tools(
    system_message=SYSTEM_MESSAGE_PREFIX, # 指定提示词前缀
    llm=llm, tools=tools, memory=memory, 
    verbose=True, # 是否打印调试日志，方便查看每个环节执行情况
    output_parser=output_parser # 
)
agent = AgentExecutor.from_agent_and_tools(
    agent=chat_agent, tools=tools, memory=memory, verbose=True,
    max_iterations=3 # 设置大模型循环最大次数，防止无限循环
)


agent('55*22的结果')



> Entering new AgentExecutor chain...
```json
{
    "action": "Calculator",
    "action_input": "55*22"
}
```
```json
{
    "action": "Calculator",
    "action_input": "55*22"
}
```
Observation: 1210
Thought:```json
{
    "action": "Final Answer",
    "action_input": "55乘以22的结果是1210"
}
```
```json
{
    "action": "Final Answer",
    "action_input": "55乘以22的结果是1210"
}
```

> Finished chain.


{'input': '55*22的结果',
 'chat_history': [HumanMessage(content='55*22的结果'),
  AIMessage(content='55乘以22的结果是1210')],
 'output': '55乘以22的结果是1210'}

In [13]:
agent('《奥本海默》的导演是谁')



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "奥本海默 导演"
}
```
```json
{
    "action": "Search",
    "action_input": "奥本海默 导演"
}
```
正在调用搜索引擎执行查询: 奥本海默 导演

Observation: 克里斯托弗·诺兰的传记片《奥本海默》 的基本设定很简单：讲述被誉为"原子弹之父"的物理学家j·罗伯特·奥本海默的故事。 但是，与这位导演的其他电影一样，该片的呈现过程绝非简单。这部电影在不同的时代之间跳跃，在一连串政府听证会中出现了一系列令人眼花缭乱的科学家、政治人士和 ... 传记电影《奥本海默》(Oppenheimer)于7月21日上映 之际，以《星际穿越》(Interstellar)和《盗梦空间》(Inception)等烧脑电影闻名的编剧兼导演克里斯托弗·诺兰，讲述了儿时的一个恐惧——不是基于科幻小说，而是基于真实的科学，即热核战争和人类灭绝的威胁。 相比诺兰儿子的身份，这次他更像是随行摄影师，而他镜头下的导演诺兰，显然充分享受着在这个既古老又先锋的国度度过的短暂时光。. 而在诺兰停留上海期间，澎湃新闻也得以对其进行了专访。. 记者就《奥本海默》剧本创作上内容的取舍、影片中出场角色的 ... 感谢环球影业提供的宝贵机会，在电影《奥本海默》上映之际，我们有幸能和诺兰导演坐在一起，面对面地聊聊关于电影创作的话题。诺兰导演为 ... 奥本海默的"300多年物理学巅峰"科研胜利遇上人性道德难题，始于过去，立于当下。 此前，诺兰四次获得导演工会奖的提名，但每一次都铩羽而归。这一次，诺兰用《奥本海默》冲击奖项，终于拿下奥斯卡最佳导演奖。
Thought:```json
{
    "action": "Final Answer",
    "action_input": "《奥本海默》的导演是克里斯托弗·诺兰。"
}
```
```json
{
    "action": "Final Answer",
    "action_input": "《奥本海默》的导演是克里斯托弗·诺兰。"
}
```

> Finished chain.


{'input': '《奥本海默》的导演是谁',
 'chat_history': [HumanMessage(content='55*22的结果'),
  AIMessage(content='55乘以22的结果是1210'),
  HumanMessage(content='《奥本海默》的导演是谁'),
  AIMessage(content='《奥本海默》的导演是克里斯托弗·诺兰。')],
 'output': '《奥本海默》的导演是克里斯托弗·诺兰。'}

##  使用LangChain内置工具PythonREPLTool

In [16]:
!pip install pinyin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pinyin: filename=pinyin-0.4.0-py3-none-any.whl size=3630476 sha256=75739bf8953b39dcd1e77ca5aace198e152beb4d368a675e4b3ab3619520235b
  Stored in directory: /root/.cache/pip/wheels/33/38/af/616fc6f154aa5bae65a1da12b22d79943434269f0468ff9b3f
Successfully built pinyin


In [37]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
langchain.debug=False
llm = ChatOpenAI(model_name='qwen1.5-110b-chat',temperature=0.1)
agent = create_python_agent(
    llm,  #使用前面一节已经加载的大语言模型
    tool=PythonREPLTool(), #使用Python交互式环境工具 REPLTool
    verbose=True #输出中间步骤
)
customer_list = ["小明","小黄","小红","小蓝","小橘","小绿",]
agent.run(f"使用pinyin拼音库将这些客户名字转换为拼音，并打印输出列表,代码中以```python开头: {customer_list}。") 




> Entering new AgentExecutor chain...
我需要使用pinyin库来将中文名字转换为拼音。首先，我需要安装pinyin库，然后使用它来处理给定的姓名列表。
Action: Python_REPL
Action Input: 
```python
from pinyin import pinyin, Style

names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']
pinyin_names = [item[0] for item in pinyin(names, style=Style.NORMAL)]
print(pinyin_names)
```
Observation: ImportError("cannot import name 'Style' from 'pinyin' (/opt/conda/lib/python3.10/site-packages/pinyin/__init__.py)")
Thought:看起来我遇到了一个ImportError，可能是因为pinyin库的版本问题或者使用方法有误。我需要调整代码，确保正确导入Style并转换拼音。
Action: Python_REPL
Action Input:
```python
from pinyin import pinyin

names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']
pinyin_names = [item for sublist in pinyin(names) for item in sublist]
print(pinyin_names)
```
Observation: TypeError("'module' object is not callable")
Thought:我再次遇到错误，这次是TypeError，这表明我调用pinyin的方式有问题。我需要确认正确的使用方法来获取每个名字的拼音列表。
Action: Python_REPL
Action Input:
```python
from pinyin import get

names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']
pinyin_names = [g

"使用pinyin库，这些客户名字转换为拼音后的列表为：['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']。"

In [38]:
## 使用debug模式
import langchain
langchain.debug=True
agent.run(f"使用pinyin拼音库将这些客户名字转换为拼音，并打印输出列表: {customer_list}") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "使用pinyin拼音库将这些客户名字转换为拼音，并打印输出列表: ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "使用pinyin拼音库将这些客户名字转换为拼音，并打印输出列表: ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I d